<a href="https://colab.research.google.com/github/kushal8697/FinalModel/blob/main/Final_of_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = '140k-real-and-fake-faces:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F501529%2F939937%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240909%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240909T055927Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D067dd3e1c366bbc3fbc150ce1f0c83400d4986a60df6e77f6404df047d9ea95c2be9c2ca4e7b2e62cb4bd07c181fe3bce079047b7a4233440750f13c170fc39e406a8f0424b6189c1fc28d2ef42e8c4a86feb43aa2b6d0073833c9df15341f32bed4aa1e3555cfa58ec673943e776cd03cfdbea4b48df00f18cf4ea314aa106f1a119d2fda087dae52c846dfe3e5ec381a0d4f668167bbc037eb4f89eaf9895bd945a7e613110c0997e204cc917e0ea06125b4a6d981d4b20ccca1cbf7bcb7cf55ddd0b122abc5207897c669755d76f026c1dddcdb6be45a6ea0465c08ac16ebadc6c6529aca4c298fc6cbcbc054918ea77a4eba8b0e2a13ef0c0cae1a9f9bcf,faceforensics:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F519884%2F956177%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240909%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240909T055927Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db4517851e300b410c23313518c1e6881ea5595395d45513855651a7da5c49f71f763c5cb2adc375f5617022ec5cf4e2b7765c42cd8686c5925d0e94ca15e4c41d7a1ea2b156b3052bdc58d95b3cb7bbe92632c96f7738e25e68f4c9f421117f14b74b59926ffa9b7171c2147ddc46fcc7cf770d85fa5a2e4fe28d47f9be95fa7d050e9989626e38f9ca14b10ff476c5b0cc443645c1e47806fb732bae445e96aa6fea889a05de2eba03bf47772e5183d9d5d34d9f970428f9b054445293d57fdb785276d224e70f16b40c94abe7932cef938f1e5b91b3a72b71b46722f455dc5d1ba5d8ee02638ded1d8c9c4df5d79f187018cfffd7a909b59dc97780c8f2770,modelos:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3382782%2F6037307%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240909%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240909T055927Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Daf44b165ab42a318a1ed4f42963e3ed2daf85cce4ff92275477f1101579dd09545fc6f55f141c86d881e31e0be0deb9ff1252bb98e9e8983070dd7efd85987d33c8d0b7d7e5933fe6ca70af503fe60d4dfa9b5d4d58abe9f584afa7e1b85e30f15cc8bf278f8f8cd6ba75d02a7050123165d9e35c7e4940895c9365e120700fbcb634a66ff01f255b90f8622813c8bd4fa3720126e61be94affe5748fe9b391978bcde0dc0c28366eccb42b84bf1a4c4a0499bb61c272392a60846d72089d7804014c3948bce0e757ef2a77df6e55ff4ddd893dc61ec976bf28afa187056f0eaf448b4fadf31ba97e16fec3ffb568c3a1a0b8f4d5bb361839864a8db16850745,convnext/tensorflow2/tiny-1k-224/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F381%2F504%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240909%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240909T055928Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dc2e18fe7ebfac168b2886998b78b199e072fe0ea61eaf86d89b811f73c3dd861f535c2f6eb5ced1756bdfac65c4842946ab30230d51ebdead166ace54a7264b99bc045641cc2acfd4bca127ae94dae19485eb3e843cbde25d4c03488a733ee26b76536ddc4c286c950da9935fdb535ea47721c9c72840d79f00dc9715a2e742eee62d8f9fab96164663de65265e09c0344a3f3d72ca9fe6a5cb569d58039ecf829fc1d5e791adf21aff0e3a19fcc581dbf10122c7c46ebe9471c23e2fd6c624b64e72d8e1c3d8ee1dcda5ed2aeab7a8967031db26424787f8cfceb2c5a69d8e01e0f0fc90f6003af734c173e331bed85445b1328c37ca63d3f66cb13c93dc312'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 4024555718 bytes downloaded
Downloaded and uncompressed: 140k-real-and-fake-faces
[==================================================] 724841395 bytes downloaded
Downloaded and uncompressed: faceforensics
[==================================================] 532033406 bytes downloaded
Downloaded and uncompressed: modelos
[==================================================] 106802255 bytes downloaded
Downloaded and uncompressed: convnext/tensorflow2/tiny-1k-224/1
Data source import complete.


## **Algorithm for Detection of DeepFake Face Images through Machine Learning

# **1. Introduction**

In [ ]:
pip install tqdm

In [ ]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.7 MB/s eta 0:00:00


In [ ]:
pip install seaborn

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [ ]:
import pandas as pd
import random
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sb
import torch
from torchvision import models
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models.vision_transformer import vit_b_16, ViT_B_16_Weights
from torchvision.utils import make_grid
from torch.autograd import Variable
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support
import shutil  ##offers high-level operation on a file like a copy, create, and remote operation on the file
import os
import warnings
warnings.filterwarnings("ignore")

code_dir = "/kaggle/working/code"
model_dir = "/kaggle/working/model"
output_dir = "/kaggle/working/output"

if not os.path.exists(code_dir):
    os.mkdir(code_dir)

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

shutil.copyfile(src="/kaggle/input/modelos/convnext.py",
                dst="/kaggle/working/code/convnext.py")
shutil.copyfile(src="/kaggle/input/modelos/convnext_tiny_1k_224_ema.pth",
                dst="/kaggle/working/model/convnext_tiny_1k_224_ema.pth")
shutil.copyfile(src="/kaggle/input/modelos/vit_b_16-c867db91.pth",
                dst="/kaggle/working/model/vit_b_16-c867db91.pth")

os.chdir("/kaggle/working/code")


from convnext import ConvNeXt


# **2. Database**

## **2.1. Network Architectures**

In [ ]:
#Instantiating the ConvNeXt model
def ConvNeXt_model():
    model_conv=ConvNeXt()
    state_dict = torch.load('/kaggle/working/model/convnext_tiny_1k_224_ema.pth')
    model_conv.load_state_dict(state_dict["model"])

    return model_conv

def ViT_model():
    model_vit=vit_b_16(pretrained=True)
    return model_vit


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
  criterion = nn.CrossEntropyLoss()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
train_acc, teste_acc, train_loss, teste_loss = [], [], [], []
train_precision, teste_precision, train_recall, teste_recall = [], [], [], []
train_f1, teste_f1 = [], []
df = pd.DataFrame(columns=['Modelo','Experimento','Epoch', 'Train ACC', 'Train Loss', 'Train F1', 'Test ACC', 'Test Loss', 'Test F1'])

# **3. Methodology**

## **3.1.  Data Augmentation**

### **3.1.1. Image Processing**

In [ ]:
def full_data_transform(model_type, data_fraction, batch_size):
    # Data transformations
    if model_type== 'convnext':
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    elif model_type == 'vit':
        transform = ViT_B_16_Weights.IMAGENET1K_V1.transforms()


    local_arquivos='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
    full_train_dataset = ImageFolder(local_arquivos + "/train", transform=transform)
    full_test_dataset = ImageFolder(local_arquivos + "/test", transform=transform)

    # Determine the number of objects to be selected
    num_train_data = int(len(full_train_dataset) * data_fraction)
    num_test_data = int(len(full_test_dataset) * data_fraction)



    # Randomly select objects for the datasets
    train_indices = random.sample(range(len(full_train_dataset)), num_train_data)
    test_indices = random.sample(range(len(full_test_dataset)), num_test_data)

    # Create datasets with randomly selected objects
    train_dataset = torch.utils.data.Subset(full_train_dataset, train_indices)
    test_dataset = torch.utils.data.Subset(full_test_dataset, test_indices)

    # Create dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=int(batch_size/2), shuffle=False)


    return train_dataloader, test_dataloader

def ft_data_transform(model_type, data_fraction, batch_size):
    # Data transformations
    if model_type== 'convnext':
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


    elif model_type == 'vit':
        transform = ViT_B_16_Weights.IMAGENET1K_V1.transforms()

    local_arquivos='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
    full_test_dataset = ImageFolder(local_arquivos + "/test", transform=transform)

    # Determine the number of objects to be selected
    num_test_data = int(len(full_test_dataset) * data_fraction)


    # Randomly select objects for the datasets
    test_indices = random.sample(range(len(full_test_dataset)), num_test_data)

    # Create datasets with randomly selected objects
    test_dataset = torch.utils.data.Subset(full_test_dataset, test_indices)

    # Create dataloaders
    test_dataloader = DataLoader(test_dataset, batch_size=int(batch_size/2), shuffle=False)

    return test_dataloader

### **3.1.2. AutoAugment**

In [ ]:
def AutoAugment_transform(model_type, train_indices, batch_size):
    if model_type== 'convnext':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.AutoAugment(), ## for automatically enhancing the diversity and quality of the training data
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    elif model_type == 'vit':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(224, interpolation=Image.BILINEAR),
            transforms.CenterCrop(224),
            transforms.AutoAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    local_arquivos='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
    augmented_train_dataset = ImageFolder(local_arquivos + "/train", transform=augmentation_transforms)
    augmented_train_dataset = torch.utils.data.Subset(augmented_train_dataset, train_indices)
    augmented_train_dataloader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True)

    return augmented_train_dataloader


### **3.1.3. RandAugment**

In [ ]:
def RandAugment_transform(model_type, train_indices, batch_size):
    if model_type== 'convnext':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.RandAugment(),#RandAugment simplifies the process by using a reduced search space. Instead of searching for the best policy from scratch
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    elif model_type == 'vit':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(224, interpolation=Image.BILINEAR),
            transforms.CenterCrop(224),
            transforms.RandAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    local_arquivos='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
    augmented_train_dataset = ImageFolder(local_arquivos + "/train", transform=augmentation_transforms)
    augmented_train_dataset = torch.utils.data.Subset(augmented_train_dataset, train_indices)
    augmented_train_dataloader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True)

    return augmented_train_dataloader


### **3.1.4. Auto+Rand Augment**

In [ ]:
def Auto_RandAugment_transform(model_type, train_indices, batch_size):
    if model_type== 'convnext':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.AutoAugment(),
            transforms.RandAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    elif model_type == 'vit':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(224, interpolation=Image.BILINEAR),
            transforms.CenterCrop(224),
            transforms.AutoAugment(),
            transforms.RandAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    local_arquivos='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
    augmented_train_dataset = ImageFolder(local_arquivos + "/train", transform=augmentation_transforms)
    augmented_train_dataset = torch.utils.data.Subset(augmented_train_dataset, train_indices)
    augmented_train_dataloader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True)

    return augmented_train_dataloader

## **3.2. Training and Testing**

In [ ]:
 # Generic training function
def train(model, dataloader, criterion, optimizer, scheduler, device, ft, epoch, exp, model_type):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    y_true,y_pred=[], []

    loop = tqdm(enumerate(dataloader), total=len(dataloader))
    for batch_idx, (images, labels) in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad() #Clears the gradients of all optimized tensors before performing backpropagation.
        if ft==False:    #Conditionally executes either fine-tuning or regular training based on the ft flag
            for param in model.parameters():
                param.requires_grad=False   #Freeze model parameters

            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.step()

            for param in model.parameters():
                param.requires_grad=True
            #After the optimization step, this loop iterates over all parameters of the model again and sets requires_grad attribute to True. This unfreezes the model's weights and biases, allowing them to be updated during subsequent iterations of training.
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        running_loss += loss.item()
        predicted = outputs.argmax(dim = 1)

        y_true.extend(labels.cpu().tolist())
        y_pred.extend(predicted.cpu().tolist())

        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        loop.set_description(f"[Epoch {(epoch+1)}]")
        loop.set_postfix(loss=loss.item())

    if scheduler:
        scheduler.step()

    train_loss = running_loss / len(dataloader.dataset)
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)

    print(f"Train Loss: {train_loss:.6f} | Train Accuracy: {(accuracy * 100):.2f}% | Train F1-Score: {f1:.6f}")

    model_name= f'model_{model_type}_params_exp_{exp}.pth'
    torch.save(model.state_dict(), os.path.join('/kaggle/working/model', model_name))

    return train_loss, accuracy, f1

In [ ]:
# Generic testing function
def test(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    y_pred, y_true= [], []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)


            running_loss += loss.item()
            predicted = outputs.argmax(dim = 1)

            y_true.extend(labels.cpu().tolist())
            y_pred.extend(predicted.cpu().tolist())

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss = running_loss / len(list(dataloader.dataset))
    test_accuracy = accuracy = accuracy_score(y_true, y_pred)
    precision, recall, test_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)

    print(f"Test Loss: {test_loss:.6f} | Test Accuracy: {(test_accuracy * 100):.2f}% | Test F1-Score: {test_f1:.6f}")
    return test_loss, test_accuracy, test_f1


In [ ]:
#Generic function for training and testing
def train_model(model_type, exp,model, train_dataloader, test_dataloader, criterion, optimizer, scheduler, device, num_epochs, ft, num):
    model=model.to(device)
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []

    for epoch in range(num_epochs):
        print('----------------------------------------------------------------------------')
        train_loss, train_accuracy, train_f1 = train(model, train_dataloader, criterion, optimizer, scheduler, device, ft, epoch, exp, model_type)
        test_loss, test_accuracy,test_f1 = test(model, test_dataloader, criterion, device)
        val=str(num)+str(epoch+1)
        df.loc[val]=[model_type, exp, epoch+1, train_accuracy, train_loss, train_f1, test_accuracy, test_loss, test_f1]
        df.to_csv('metricas.csv', index = False)
        print('\n')



    return train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1


## **3.3. Experimental Protocol**

Applying experimental protocol to scenarios resulting from permutations of Data Augmentations (AutoAugment and RandAugment) and Fine-Tuning:
1. Without Fine-Tuning
2. With Fine-Tuning
3. Without Fine-Tuning and with AutoAugment
4. With Fine-Tuning and with AutoAugment
5. Without Fine-Tuning and with RandAugment
6. With Fine-Tuning and with RandAugment
7. Without Fine-Tuning and with both AutoAugment and RandAugment
8. With Fine-Tuning and with both AutoAugment and RandAugment

In [ ]:
def run_scenario(model_type, scenario, data_fraction, num_epochs=10, batch_size=32, learning_rate=0.001):
    num_classes = 2
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    num_train_data = int((100000) * data_fraction)
    num_test_data = int((20000) * data_fraction)

    train_indices = random.sample(range(65000), num_train_data)
    test_indices = random.sample(range(20000), num_test_data)

    print(f'Number of training objects: {num_train_data}')
    print(f'Number of test objects: {num_test_data}')
    print("============================================================================")

    if scenario == 1:

        train_dataloader, test_dataloader= full_data_transform(model_type, data_fraction, batch_size)


        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=1

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=9
        else:
            raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experimento 1 - ConvNeXt (Sem Fine-Tuning)")
        elif model_type == 'vit':
            print("Experimento 1 - ViT (Sem Fine-Tuning)")

        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,1, model, train_dataloader, test_dataloader,
                                                                                    criterion, optimizer, scheduler, device, 1, False, num)
    elif scenario == 2:

        train_dataloader, test_dataloader= full_data_transform(model_type, data_fraction, batch_size)


        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=2

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=10
        else:
            raise ValueError("O parâmetro 'model_type' deve ser 'convnext' ou 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
             print("Experiment 2 - ConvNeXt (With Fine-Tuning)")
        elif model_type == 'vit':
            print("Experiment 2 - ViT (With Fine-Tuning)")

        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,2, model, train_dataloader, test_dataloader,
                                                                                    criterion, optimizer, scheduler, device, num_epochs, True, num)
    elif scenario == 3:

        augmented_train_dataloader = AutoAugment_transform(model_type, train_indices, batch_size)
        test_dataloader=ft_data_transform(model_type, data_fraction, batch_size)


        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=3

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=11
        else:
            raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experiment 3 - ConvNeXt (No Fine-Tuning with AutoAugment")
        elif model_type == 'vit':
            print("Experiment 3 - ViT (No Fine-Tuning with AutoAugment)")


        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,3, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, 1, False, num)

        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Replace num_classes with the correct number of classes
            num=4

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=12
        else:
            raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experiment 4 - ConvNeXt (With Fine-Tuning and with AutoAugment)")
        elif model_type == 'vit':
            print("Experiment 4 - ViT (With Fine-Tuning and with AutoAugment)")

        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,4, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, num_epochs, True, num)

    elif scenario == 4:
        augmented_train_dataloader = RandAugment_transform(model_type, train_indices, batch_size)
        test_dataloader=ft_data_transform(model_type, data_fraction, batch_size)

        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=5

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=13
        else:
            raise ValueError("O parâmetro 'model_type' deve ser 'convnext' ou 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experiment 5 - ConvNeXt (No Fine-Tuning and with RandAugment)")
        elif model_type == 'vit':
            print("Experiment 5 - ViT (No Fine-Tuning and with RandAugment)")

        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,5, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, 1, False, num)

        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  #Replace num_classes with the correct number of classes
            num=6

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=14
        else:
            raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experiment 6 - ConvNeXt (With Fine-Tuning and with RandAugment)")
        elif model_type == 'vit':
            print("Experiment 6 - ViT (With Fine-Tuning and with RandAugment)")


        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,6, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, num_epochs, True, num)

    elif scenario == 5:
        augmented_train_dataloader = Auto_RandAugment_transform(model_type, train_indices, batch_size)
        test_dataloader=ft_data_transform(model_type, data_fraction, batch_size)

        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=7

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=15
        else:
            raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experiment 7 - ConvNeXt (No Fine-Tuning and with RandAugment and AutoAugment)")
        elif model_type == 'vit':
            print("Experiment 7 - ViT (No Fine-Tuning and with RandAugment and AutoAugment)")

        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,7, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, 1, False, num)

        # Select the model
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)
            num=8

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=16
        else:
            raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experiment 8 - ConvNeXt (With Fine-Tuning and with RandAugment and AutoAugment)")
        elif model_type == 'vit':
            print("Experiment 8 - ViT (With Fine-Tuning and with RandAugment and AutoAugment)")

        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,8, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, num_epochs, True, num)
    else:
        raise ValueError("The 'model_type' parameter must be 'convnext' or 'vit'.")



## **3.4. Performance Evaluation**

In [ ]:
import matplotlib.pyplot as plt

def plot_experiment_graphs(df, experiment_number):
    experiment_df = df[df['Experimento'] == experiment_number]

    #  Plot epoch x train F1
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Train F1'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Train F1')
    plt.title(f'Experiment {experiment_number} - Train F1')
    plt.tight_layout()
    plt.show()

    # Plot epoch x train accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Train ACC'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Train Accuracy')
    plt.title(f'Experiment {experiment_number} - Train Accuracy')
    plt.tight_layout()
    plt.show()

    # Plot epoch x train loss
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Train Loss'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Train Loss')
    plt.title(f'Experimento {experiment_number} - Trai loss')
    plt.tight_layout()
    plt.show()

    # Plot epoch x test F1
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Test F1'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Test F1')
    plt.title(f'Experiment {experiment_number} - Test F1')
    plt.tight_layout()
    plt.show()

    # Plot epoch x test accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Test ACC'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Test Accuracy')
    plt.title(f'Experiment {experiment_number} - Test Accuracy')
    plt.tight_layout()
    plt.show()

    # Plot epoch x test loss
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Test Loss'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Test loss')
    plt.title(f'Experimento {experiment_number} - Test loss')
    plt.tight_layout()
    plt.show()

def plot_general_graphs(df):
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

    # Plot epoch x train F1
    ax1 = axes[0, 0]
    for experiment_number in df['Experimento'].unique():
        experiment_df = df[df['Experimento'] == experiment_number]
        ax1.plot(experiment_df['Epoch'], experiment_df['Train F1'], marker='o', label=f'Experimento {experiment_number}')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Train F1')
    ax1.set_title('Overall - Train F1')
    ax1.legend()

    # Plot epoch x train loss
    ax2 = axes[0, 1]
    for experiment_number in df['Experiment'].unique():
        experiment_df = df[df['Experiment'] == experiment_number]
        ax2.plot(experiment_df['Epoch'], experiment_df['Train Loss'], marker='o', label=f'Experiment {experiment_number}')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Train Loss')
    ax2.set_title('Overall - Train Loss')
    ax2.legend()

    # Plot epoch x test F1
    ax3 = axes[1, 0]
    for experiment_number in df['Experimento'].unique():
        experiment_df = df[df['Experimento'] == experiment_number]
        ax3.plot(experiment_df['Epoch'], experiment_df['Test F1'], marker='o', label=f'Experimento {experiment_number}')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Test F1')
    ax3.set_title('Overall - Test F1')
    ax3.legend()

    # Plot epoch x test loss
    ax4 = axes[1, 1]
    for experiment_number in df['Experiment'].unique():
        experiment_df = df[df['Experiment'] == experiment_number]
        ax4.plot(experiment_df['Epoch'], experiment_df['Test Loss'], marker='o', label=f'Experimento {experiment_number}')
    ax4.set_xlabel('Epoch')
    ax4.set_ylabel('Test Loss')
    ax4.set_title('Overall - Test Loss')
    ax4.legend()

    plt.tight_layout()
    plt.show()


In [ ]:

run_scenario('convnext',1,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)


Number of training objects: 10000
Number of test objects: 2000
Experimento 1 - ConvNeXt (Sem Fine-Tuning)
----------------------------------------------------------------------------


[Epoch 1]: 100%|██████████| 625/625 [01:46<00:00,  5.89it/s, loss=0.71]


Train Loss: 0.043574 | Train Accuracy: 51.21% | Train F1-Score: 0.512008
Test Loss: 0.087375 | Test Accuracy: 50.55% | Test F1-Score: 0.505203




In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(df)

      Modelo  Experimento  Epoch  Train ACC  Train Loss  Train F1  Test ACC  \
11  convnext            1      1     0.5121    0.043574  0.512008    0.5055   

    Test Loss   Test F1  
11   0.087375  0.505203  


In [ ]:
run_scenario('convnext',2,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

Number of training objects: 10000
Number of test objects: 2000
Experiment 2 - ConvNeXt (With Fine-Tuning)
----------------------------------------------------------------------------


[Epoch 1]: 100%|██████████| 625/625 [03:39<00:00,  2.85it/s, loss=0.000668]


Train Loss: 0.006969 | Train Accuracy: 95.44% | Train F1-Score: 0.954398
Test Loss: 0.001356 | Test Accuracy: 99.65% | Test F1-Score: 0.996490


----------------------------------------------------------------------------


[Epoch 2]: 100%|██████████| 625/625 [03:24<00:00,  3.06it/s, loss=0.00185]


Train Loss: 0.000660 | Train Accuracy: 99.64% | Train F1-Score: 0.996400
Test Loss: 0.004834 | Test Accuracy: 98.65% | Test F1-Score: 0.986476


----------------------------------------------------------------------------


...

Train Loss: 0.000018 | Train Accuracy: 100.00% | Train F1-Score: 1.000000
Test Loss: 0.000472 | Test Accuracy: 99.85% | Test F1-Score: 0.998496


----------------------------------------------------------------------------


[Epoch 9]: 100%|██████████| 625/625 [03:23<00:00,  3.07it/s, loss=0.000316]


Train Loss: 0.000018 | Train Accuracy: 100.00% | Train F1-Score: 1.000000
Test Loss: 0.000472 | Test Accuracy: 99.85% | Test F1-Score: 0.998496


----------------------------------------------------------------------------


[Epoch 10]: 100%|██████████| 625/625 [03:23<00:00,  3.08it/s, loss=9.49e-5]


Train Loss: 0.000018 | Train Accuracy: 100.00% | Train F1-Score: 1.000000
Test Loss: 0.000472 | Test Accuracy: 99.85% | Test F1-Score: 0.998496




In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)


                      Epoch  Train ACC  Train Loss  Train F1  Test ACC  \
Modelo   Experimento                                                     
convnext 1              1.0    0.51210    0.043574  0.512008    0.5055   
         2              5.5    0.99502    0.000795  0.995020    0.9971   

                      Test Loss   Test F1  
Modelo   Experimento                       
convnext 1             0.087375  0.505203  
         2             0.001019  0.997093  


In [ ]:
run_scenario('convnext',3,0.1, num_epochs=3, batch_size=16, learning_rate=0.0001)

Number of training objects: 10000
Number of test objects: 2000
Experiment 3 - ConvNeXt (No Fine-Tuning with AutoAugment
----------------------------------------------------------------------------


[Epoch 1]: 100%|██████████| 625/625 [01:33<00:00,  6.67it/s, loss=0.659]


Train Loss: 0.042030 | Train Accuracy: 59.64% | Train F1-Score: 0.486636
Test Loss: 0.087630 | Test Accuracy: 50.15% | Test F1-Score: 0.475536


Experiment 4 - ConvNeXt (With Fine-Tuning and with AutoAugment)
----------------------------------------------------------------------------


[Epoch 1]: 100%|██████████| 625/625 [03:33<00:00,  2.93it/s, loss=0.00345]


Train Loss: 0.011989 | Train Accuracy: 92.05% | Train F1-Score: 0.879116
Test Loss: 0.010821 | Test Accuracy: 96.85% | Test F1-Score: 0.968494


----------------------------------------------------------------------------


[Epoch 2]: 100%|██████████| 625/625 [03:33<00:00,  2.93it/s, loss=0.0648]


Train Loss: 0.002986 | Train Accuracy: 98.32% | Train F1-Score: 0.976241
Test Loss: 0.001942 | Test Accuracy: 99.65% | Test F1-Score: 0.996499


----------------------------------------------------------------------------


[Epoch 3]: 100%|██████████| 625/625 [03:33<00:00,  2.93it/s, loss=0.00141]


Train Loss: 0.001164 | Train Accuracy: 99.38% | Train F1-Score: 0.991243
Test Loss: 0.001057 | Test Accuracy: 99.85% | Test F1-Score: 0.998500




In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)


                      Epoch  Train ACC  Train Loss  Train F1  Test ACC  \
Modelo   Experimento                                                     
convnext 1              1.0   0.512100    0.043574  0.512008  0.505500   
         2              5.5   0.995020    0.000795  0.995020  0.997100   
         3              1.0   0.596400    0.042030  0.486636  0.501500   
         4              2.0   0.965833    0.005379  0.948867  0.987833   

                      Test Loss   Test F1  
Modelo   Experimento                       
convnext 1             0.087375  0.505203  
         2             0.001019  0.997093  
         3             0.087630  0.475536  
         4             0.004607  0.987831  


In [ ]:
run_scenario('convnext',4,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

Number of training objects: 10000
Number of test objects: 2000
Experiment 5 - ConvNeXt (No Fine-Tuning and with RandAugment)
----------------------------------------------------------------------------


[Epoch 1]: 100%|██████████| 625/625 [01:34<00:00,  6.64it/s, loss=0.738]


Train Loss: 0.047590 | Train Accuracy: 29.04% | Train F1-Score: 0.284209
Test Loss: 0.088286 | Test Accuracy: 47.15% | Test F1-Score: 0.433656


Experiment 6 - ConvNeXt (With Fine-Tuning and with RandAugment)
----------------------------------------------------------------------------


[Epoch 1]: 100%|██████████| 625/625 [03:35<00:00,  2.90it/s, loss=0.00739]


Train Loss: 0.010994 | Train Accuracy: 92.50% | Train F1-Score: 0.888414
Test Loss: 0.003007 | Test Accuracy: 99.10% | Test F1-Score: 0.991000


----------------------------------------------------------------------------


...

Train Loss: 0.000187 | Train Accuracy: 99.94% | Train F1-Score: 0.999164
Test Loss: 0.001345 | Test Accuracy: 99.50% | Test F1-Score: 0.994999


----------------------------------------------------------------------------


[Epoch 9]: 100%|██████████| 625/625 [03:36<00:00,  2.89it/s, loss=0.000319]


Train Loss: 0.000266 | Train Accuracy: 99.87% | Train F1-Score: 0.998188
Test Loss: 0.001340 | Test Accuracy: 99.50% | Test F1-Score: 0.994999


----------------------------------------------------------------------------


[Epoch 10]: 100%|██████████| 625/625 [03:36<00:00,  2.89it/s, loss=0.00375]


Train Loss: 0.000262 | Train Accuracy: 99.87% | Train F1-Score: 0.998189
Test Loss: 0.001343 | Test Accuracy: 99.50% | Test F1-Score: 0.994999




In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

                      Epoch  Train ACC  Train Loss  Train F1  Test ACC  \
Modelo   Experimento                                                     
convnext 1              1.0   0.512100    0.043574  0.512008  0.505500   
         2              5.5   0.995020    0.000795  0.995020  0.997100   
         3              1.0   0.596400    0.042030  0.486636  0.501500   
         4              2.0   0.965833    0.005379  0.948867  0.987833   
         5              1.0   0.290400    0.047590  0.284209  0.471500   
         6              5.5   0.990120    0.001578  0.985520  0.994400   

                      Test Loss   Test F1  
Modelo   Experimento                       
convnext 1             0.087375  0.505203  
         2             0.001019  0.997093  
         3             0.087630  0.475536  
         4             0.004607  0.987831  
         5             0.088286  0.433656  
         6             0.001685  0.994399  


In [ ]:
run_scenario('convnext',5,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

Number of training objects: 10000
Number of test objects: 2000
Experiment 7 - ConvNeXt (No Fine-Tuning and with RandAugment and AutoAugment)
----------------------------------------------------------------------------


[Epoch 1]: 100%|██████████| 625/625 [01:43<00:00,  6.03it/s, loss=0.726]


Train Loss: 0.045498 | Train Accuracy: 33.74% | Train F1-Score: 0.336746
Test Loss: 0.088437 | Test Accuracy: 46.55% | Test F1-Score: 0.422965


Experiment 8 - ConvNeXt (With Fine-Tuning and with RandAugment and AutoAugment)
----------------------------------------------------------------------------


[Epoch 1]: 100%|██████████| 625/625 [03:45<00:00,  2.77it/s, loss=0.311]


Train Loss: 0.014590 | Train Accuracy: 90.19% | Train F1-Score: 0.850568
Test Loss: 0.007585 | Test Accuracy: 98.25% | Test F1-Score: 0.982499


----------------------------------------------------------------------------


...

Train Loss: 0.001060 | Train Accuracy: 99.38% | Train F1-Score: 0.991360
Test Loss: 0.001265 | Test Accuracy: 99.45% | Test F1-Score: 0.994500


----------------------------------------------------------------------------


[Epoch 9]: 100%|██████████| 625/625 [03:45<00:00,  2.77it/s, loss=0.0725]


Train Loss: 0.001078 | Train Accuracy: 99.49% | Train F1-Score: 0.992888
Test Loss: 0.001260 | Test Accuracy: 99.45% | Test F1-Score: 0.994500


----------------------------------------------------------------------------


[Epoch 10]: 100%|██████████| 625/625 [03:45<00:00,  2.77it/s, loss=0.00536]


Train Loss: 0.000853 | Train Accuracy: 99.56% | Train F1-Score: 0.993872
Test Loss: 0.001259 | Test Accuracy: 99.45% | Test F1-Score: 0.994500




In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

                      Epoch  Train ACC  Train Loss  Train F1  Test ACC  \
Modelo   Experimento                                                     
convnext 1              1.0   0.512100    0.043574  0.512008  0.505500   
         2              5.5   0.995020    0.000795  0.995020  0.997100   
         3              1.0   0.596400    0.042030  0.486636  0.501500   
         4              2.0   0.965833    0.005379  0.948867  0.987833   
         5              1.0   0.290400    0.047590  0.284209  0.471500   
         6              5.5   0.990120    0.001578  0.985520  0.994400   
         7              1.0   0.337400    0.045498  0.336746  0.465500   
         8              5.5   0.982380    0.002851  0.974155  0.992600   

                      Test Loss   Test F1  
Modelo   Experimento                       
convnext 1             0.087375  0.505203  
         2             0.001019  0.997093  
         3             0.087630  0.475536  
         4             0.004607  0.9878

In [ ]:
df.to_csv('metricas.csv', index = False)

In [ ]:
from tensorflow.keras.models import sequential

In [ ]:
model.save(my_model.h5)

NameError: name 'model' is not defined